# Required imports

In [ ]:
import pandas as pd
import scipy.stats as stats
import scikit_posthocs as sp
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns

# Load results

In [ ]:
methods = ['LBP', 'HOG', 'FN', 'CNN']
metrics   = ["FAR", "FRR", "Precision", "Recall", "Time"]
perturbations = [
    "original", "brightness_down", "brightness_up", "contrast_down", "contrast_up",
    "saturation_down", "saturation_up", "rotate_15", "rotate_-15", "gaussian_blur", "color_shift"
]

long_frames = []

for m in methods:
    g_path = f"{m}_general_statistics.csv"
    df_g   = pd.read_csv(g_path)
    df_g['Method']       = m
    df_g['Perturbation'] = None        # no perturbation

    df_g = df_g.rename(columns=str.capitalize)

    df_g_long = df_g.melt(
        id_vars=['Method', 'Perturbation'],
        value_vars=metrics, # flattening by metrics
        var_name='Metric',
        value_name='Value'
    )
    long_frames.append(df_g_long)

    p_path = f"{m}_perturbation_statistics.csv"
    df_p   = pd.read_csv(p_path)
    df_p['Method'] = m

    df_p = df_p.rename(columns=str.capitalize)

    df_p_long = df_p.melt(
        id_vars=['Method', 'Perturbation'],   # columns already exists in file
        value_vars=['far', 'frr', 'precision', 'recall'], # there is no "Time" for perturbations
        var_name='Metric',
        value_name='Value'
    )
    long_frames.append(df_p_long)

all_results = pd.concat(long_frames, ignore_index=True)
all_results.to_csv("all_results_long.csv", index=False)

In [ ]:
methods = ['LBP', 'HOG', 'FN', 'CNN']
metrics   = ["FAR", "FRR", "Precision", "Recall", "Time"]
perturbations = [
    "original", "brightness_down", "brightness_up", "contrast_down", "contrast_up",
    "saturation_down", "saturation_up", "rotate_15", "rotate_-15", "gaussian_blur", "color_shift"
]

general_dfs = []
for m in methods:
    df = pd.read_csv(f"{m}_general_statistics.csv")
    df["Method"] = m
    general_dfs.append(df)
df_general_all = pd.concat(general_dfs)

perturbation_dfs = []
for m in methods:
    df = pd.read_csv(f"{m}_perturbation_statistics.csv")
    df["Method"] = m
    perturbation_dfs.append(df)
df_perturb_all = pd.concat(perturbation_dfs)

df_general_long = pd.melt(df_general_all, id_vars=["Method"], value_vars=methods, var_name="Metric", value_name="Value")
df_perturb_long = pd.melt(df_perturb_all, id_vars=["Method", "perturbation"], value_vars=methods[:4], var_name="Metric", value_name="Value")

# Statistical analysis

In [ ]:
alpha = 0.05

In [ ]:
def run_general_tests(df):
    results = []
    for metric in metrics:
        data = df[df["Metric"] == metric]

        # normality
        pvals = [stats.shapiro(data[data["Method"] == m]["Value"]).pvalue for m in methods]
        normal = all(p > alpha for p in pvals)

        # homogeneity of variance
        grouped = [data[data["Method"] == m]["Value"] for m in methods]
        _, p_levene = stats.levene(*grouped)
        equal_var = p_levene > alpha

        if normal  and equal_var:
            stat, p = stats.f_oneway(*(data[data["Method"] == m]["Value"] for m in methods))
            test_type = "ANOVA"
            # post hoc Tukey
            tukey = pairwise_tukeyhsd(data["Value"], data["Method"])
            posthoc_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])

        elif normal and not equal_var:
            model = ols("Value ~ Method", data=data).fit()
            anova_table = sm.stats.anova_lm(model, typ=2, robust="hc3")
            stat = anova_table["F"][0]
            p = anova_table["PR(>F)"][0]
            test_type = "Welch ANOVA"
            # post hoc Games Howell
            posthoc_df = pg.pairwise_gameshowell(dv="Value", between="Method", data=data)

        else:
            stat, p = stats.kruskal(*(data[data["Method"] == m]["Value"] for m in methods))
            test_type = "Kruskal-Wallis"
            # post hoc Dunn
            posthoc_df = sp.posthoc_dunn(data, val_col="Value", group_col="Method", p_adjust="bonferroni")
            posthoc_df.reset_index(inplace=True)

        results.append({
            "Metric": metric,
            "Test": test_type,
            "Stat": stat,
            "p-value": p
        })

        posthoc_df.to_csv(f"posthoc_general_{metric}.csv", index=False)

    pd.DataFrame(results).to_csv("general_test_results.csv", index=False)
    return results, posthoc_df

In [ ]:
def run_perturb_tests(df):
    all_results = []
    for perturb in perturbations:
        for metric in metrics[:4]:  # without time
            data = df[(df["perturbation"] == perturb) & (df["Metric"] == metric)]

            pvals = [stats.shapiro(data[data["Method"] == m]["Value"]).pvalue for m in methods]
            normal = all(p > alpha for p in pvals)

            # homogeneity of variance
            method_groups = [data[data["Method"] == m]["Value"] for m in methods]
            p_levene = stats.levene(*method_groups).pvalue
            equal_var = p_levene > alpha

            if normal and equal_var:
                stat, p = stats.f_oneway(*(data[data["Method"] == m]["Value"] for m in methods))
                test_type = "ANOVA"
                tukey = pairwise_tukeyhsd(data["Value"], data["Method"])
                posthoc_df = pd.DataFrame(data=tukey._results_table.data[1:], columns=tukey._results_table.data[0])
            elif normal and not equal_var:
                model = pg.welch_anova(dv="Value", between="Method", data=data)
                stat = model["F"].values[0]
                p = model["p-unc"].values[0]
                test_type = "Welch ANOVA"
                posthoc_df = pg.pairwise_gameshowell(dv="Value", between="Method", data=data)
            else:
                stat, p = stats.kruskal(*(data[data["Method"] == m]["Value"] for m in methods))
                test_type = "Kruskal-Wallis"
                posthoc_df = sp.posthoc_dunn(data, val_col="Value", group_col="Method", p_adjust="bonferroni")
                posthoc_df.reset_index(inplace=True)

            all_results.append({
                "Perturbation": perturb,
                "Metric": metric,
                "Test": test_type,
                "Stat": stat,
                "p-value": p
            })

            posthoc_df.to_csv(f"posthoc_{perturb}_{metric}.csv", index=False)

    pd.DataFrame(all_results).to_csv("perturbation_test_results.csv", index=False)

    return all_results, posthoc_df

# Results presentation

## Evaluation Metrics Summary

### General Metrics

In [ ]:
for metric in metrics:
    df_metric = df_general_all[["Method", metric]].copy()
    df_metric = df_metric.rename(columns={metric: "Value"})

    summary = df_metric.groupby("Method")["Value"].agg(
        mean="mean",
        std="std",
        min="min",
        q1=lambda x: x.quantile(0.25),
        median="median",
        q3=lambda x: x.quantile(0.75),
        max="max"
    ).round(4).reset_index()

    print(f"\n Metric: {metric}")
    print(summary.to_string(index=False))

### Metrics for perturbations

In [ ]:
for metric in metrics[:4]:
    for pert in perturbations:
        df_subset = df_perturb_all[df_perturb_all["perturbation"] == pert]
        df_metric = df_subset[["Method", metric]].copy()
        df_metric = df_metric.rename(columns={metric: "Value"})

        summary = df_metric.groupby("Method")["Value"].agg(
            mean="mean",
            std="std",
            min="min",
            q1=lambda x: x.quantile(0.25),
            median="median",
            q3=lambda x: x.quantile(0.75),
            max="max"
        ).round(4).reset_index()

        print(f"\n Perturbation: {pert} | Metric: {metric}")
        print(summary.to_string(index=False))

## Statistical analysis (results)

In [ ]:
res, posthocs = run_general_tests(df_general_long)
print(10*"-", "  RESULTS GENERAL ", 10*"-")
print(res)
print(10*"-", " POST-HOC RESULTS ", 10*"-")
print(posthocs)

In [ ]:
res_per, posthocs_per = run_perturb_tests(df_perturb_long)
print(10*"-", " RESULTS PERTURBATION ", 10*"-")
print(res_per)
print(10*"-", "   POST-HOC RESULTS   ", 10*"-")
print(posthocs_per)

## Visualization of Results

### Histograms

In [ ]:
class_metrics = ["FAR", "FRR", "Precision", "Recall"]

agg_class = (
    df_general_long[df_general_long["Metric"].isin(class_metrics)]
    .groupby(["Method", "Metric"])["Value"]
    .agg(mean="mean", std="std")
    .reset_index()
)

plt.figure(figsize=(8, 5))
sns.barplot(
    data=agg_class,
    x="Method",
    y="mean",
    hue="Metric",
    palette="Set2",
    capsize=.15,
    errcolor="black",
    errwidth=1,
)
for idx, row in agg_class.iterrows():
    x_pos = list(methods).index(row["Method"]) + (-0.3 + 0.2*class_metrics.index(row["Metric"]))
    plt.errorbar(x=x_pos, y=row["mean"], yerr=row["std"], fmt='none',
                 ecolor='black', capsize=4, linewidth=1)

plt.title("Model Metrics Comparison")
plt.ylabel("Value")
plt.tight_layout()
plt.savefig("plots/bar_metrics.svg", format="svg")
plt.show()

In [ ]:
time_df = (
    df_general_long[df_general_long["Metric"] == "Time"]
    .groupby("Method")["Value"]
    .agg(mean="mean", std="std")
    .reset_index()
)

plt.figure(figsize=(6, 4))
sns.barplot(
    data=time_df, x="Method", y="mean",
    color="lightsteelblue", capsize=.2, errcolor="black", errwidth=1
)
plt.errorbar(x=range(len(time_df)), y=time_df["mean"], yerr=time_df["std"],
             fmt='none', ecolor='black', capsize=4, linewidth=1)
plt.title("Time of extraction")
plt.ylabel("Time [s]")
plt.tight_layout()
plt.savefig("plots/time.svg", format="svg")
plt.show()

### Heatmaps

In [ ]:
for metric in metrics:
    pivot = (
        df_perturb_long[df_perturb_long["Metric"].str.lower() == metric.lower()]
        .pivot_table(index="perturbation", columns="Method", values="Value", aggfunc="mean")
        .reindex(index=perturbations)
    )
    plt.figure(figsize=(8, 5))
    sns.heatmap(pivot, annot=True, fmt=".2f", cmap="Blues",
                cbar_kws={"label": metric})
    plt.title(f"{metric} - the impact of all perturbations")
    plt.xlabel("Method")
    plt.ylabel("Perturbation")
    plt.tight_layout()
    plt.savefig(f"plots/heatmap_perturbations_{metric}.svg", format="svg")
    plt.show()